In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-01-24_12:24
E:\RIS\Staff Folders\Samuel\Requests\SH\Safety_Data_Viewer


In [4]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')



Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-24 12:24:42


In [5]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-24 12:24:42


### Initial Table Containing All NYSDOT Information for Study Stretch. 

In [ ]:
"""

--SIP 950----------------------------------------------------------------------------------------------------------

DROP TABLE IF EXISTS sam_crashes_950; 

CREATE TABLE sam_crashes_950 AS 

SELECT distinct nys_a.*
FROM archive."2019_11_13_nysdot_all" nys_a
JOIN archive."18d.2019-11-13_lion" lion
on nys_a.mft = lion.mft
WHERE nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
and lion.segmentid::int in (82370,83437,83441,83525,83527,83529,83532,83533,83535,83537,83618,83620,83622,83624,111956,113871,145917,145918,145919,145920,148820,148821,148836,148837,148838,148839,148841,149046,149047,149048,149049,149050,149051,149053,149054,149056,149057,149058,149059,149060
                           ,149061,149062,149063,149064,149065,149066,149067,149068,149069,149070,149071,149072,149073,152926,153046,153132,153189,153829,153959,153986,154019,177087,177146,177148,177481,177531,177533,177535,177537,177539,177541,184774,234611,234612,239109,239110,9008651,9008652,9008661,9008665,9008666,9008668)


UNION


SELECT nys_a.*
FROM archive."2019_11_13_nysdot_all" nys_a
WHERE masterid in (SELECT masteridFROM mid
                   FROM archive."18d.2019-11-13_lion" lion
                   WHERE mft in (SELECT mft
                                 FROM archive."18d.2019-11-13_lion" lion
                                 WHERE lion.segmentid::int in (82370,83437,83441,83525,83527,83529,83532,83533,83535,83537,83618,83620,83622,83624,111956,113871,145917,145918,145919,145920,148820,148821,148836,148837,148838,148839,148841,149046,149047,149048,149049,149050,149051,149053,149054,149056,149057,149058,149059,149060
                                                              ,149061,149062,149063,149064,149065,149066,149067,149068,149069,149070,149071,149072,149073,152926,153046,153132,153189,153829,153959,153986,154019,177087,177146,177148,177481,177531,177533,177535,177537,177539,177541,184774,234611,234612,239109,239110,9008651,9008652,9008661,9008665,9008666,9008668)

                                 )


                   UNION 

                   SELECT masteridto mid
                   FROM archive."18d.2019-11-13_lion" lion
                   WHERE mft in (SELECT mft
                                 FROM archive."18d.2019-11-13_lion" lion
                                 WHERE lion.segmentid::int in (82370,83437,83441,83525,83527,83529,83532,83533,83535,83537,83618,83620,83622,83624,111956,113871,145917,145918,145919,145920,148820,148821,148836,148837,148838,148839,148841,149046,149047,149048,149049,149050,149051,149053,149054,149056,149057,149058,149059,149060
                                                              ,149061,149062,149063,149064,149065,149066,149067,149068,149069,149070,149071,149072,149073,152926,153046,153132,153189,153829,153959,153986,154019,177087,177146,177148,177481,177531,177533,177535,177537,177539,177541,184774,234611,234612,239109,239110,9008651,9008652,9008661,9008665,9008666,9008668)
                      )
)
and  nys_a.case_yr>= 2013 and nys_a.case_yr<=2017;

GRANT ALL on sam_crashes_950 to public;

"""

### Initial Table Containing All Fatality Information for Study Stretch. 

In [5]:
"""
--Fatals---------------------------
DROP TABLE IF EXISTS sam_fatalities_950; 

CREATE TABLE sam_fatalities_950 as 

SELECT segmentid, nodeidFROM, nodeidto 
FROM archive."18d.2019-11-13_lion" lion
WHERE lion.mft in (SELECT mft
                   FROM archive."18d.2019-11-13_lion" 
                   WHERE segmentid::int in (82370,83437,83441,83525,83527,83529,83532,83533,83535,83537,83618,83620,83622,83624,111956,113871,145917,145918,145919,145920,148820,148821,148836,148837,148838,148839,148841,149046,149047,149048,149049,149050,149051,149053,149054,149056,149057,149058,149059,149060
                                           ,149061,149062,149063,149064,149065,149066,149067,149068,149069,149070,149071,149072,149073,152926,153046,153132,153189,153829,153959,153986,154019,177087,177146,177148,177481,177531,177533,177535,177537,177539,177541,184774,234611,234612,239109,239110,9008651,9008652,9008661,9008665,9008666,9008668)
          )

"""

### 1. Sip 950
#### Created table in Pgsql containing all nysdot_all data for Sip: 950 - 111TH ST to 108TH STT

In [7]:
sip_950= cdb.dfquery("""SELECT * FROM sam_crashes_950""")

In [8]:
ksi_miles = cdb.dfquery("""


--KSI by Miles----------------------------------------------------------

WITH miles as(
SELECT sum(len)/5280 len from (
SELECT DISTINCT segmentid, st_length(geom) len
FROM archive."18d.2019-11-13_lion" lion
WHERE lion.mft in (SELECT mft
                   FROM archive."18d.2019-11-13_lion" 
                   WHERE segmentid::int in (82370,83437,83441,83525,83527,83529,83532,83533,83535,83537,83618,83620,83622,83624,111956,113871,145917,145918,145919,145920,148820,148821,148836,148837,148838,148839,148841,149046,149047,149048,149049,149050,149051,149053,149054,149056,149057,149058,149059,149060
                                           ,149061,149062,149063,149064,149065,149066,149067,149068,149069,149070,149071,149072,149073,152926,153046,153132,153189,153829,153959,153986,154019,177087,177146,177148,177481,177531,177533,177535,177537,177539,177541,184774,234611,234612,239109,239110,9008651,9008652,9008661,9008665,9008666,9008668)
                  )
and rb_layer in ('G','B')
) stretch
)

, ksi as(SELECT sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) si
         FROM sam_crashes_950 )

, fat as(
SELECT ((SELECT count(id_) as "Fatalities"
         FROM public.fatality_nycdot_current fat
         WHERE fat.segmentid in (SELECT segmentid FROM sam_fatalities_950)
         and date_part('year',acdate) between 2013 and 2019) +

         -- All fatalities on intersections of stretch
        (SELECT count(id_) as "Fatalities"
         FROM public.fatality_nycdot_current fat
         WHERE fat.nodeid::int in (SELECT distinct nodeid::int FROM(
                                   SELECT nodeidFROM nodeid FROM sam_fatalities_950 
                                   UNION
                                   SELECT nodeidto nodeid FROM sam_fatalities_950) x)
         and date_part('year',acdate) between 2013 and 2019) ) as k

    )

SELECT miles.len "LENGTH", (SELECT si + k FROM ksi, fat)/len KSI
FROM miles

""")


In [9]:
ksi_miles

,LENGTH,ksi
0,2.828852,7.070006


### Injury Summary

In [14]:
inj_sum = cdb.dfquery("""



WITH data as(
SELECT "Type", sum("Total Injuries") "Total Injuries", sum("Severe Injuries") "Severe Injuries", sum("Fatalities") "Fatalities", sum("Severe Injuries"  + "Fatalities") as "KSI"
FROM (SELECT * FROM (
      --Injuries/Severity By Mode
      SELECT  CASE WHEN accd_type_int = 1 then '1. ped'
                   WHEN accd_type_int = 2 then '2. bicycle'
                   WHEN accd_type_int = 3 then '3. mvo' end as "Type"
             ,sum(num_of_inj) as "Total Injuries"
             ,sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Severe Injuries"
             ,0 as "Fatalities"
      FROM sam_crashes_950 
      --JOIN archive."2019_11_13_nysdot_vehicle" nys_v_age
      --     on sam_crashes_950.crashid = nys_v_age.crashid
      --WHERE nys_v_age.age::int between 1 and 17
      GROUP BY  accd_type_int
      ORDER BY  accd_type_int
    ) x

    UNION


    -- All fatalities on corridors of stretch
    SELECT CASE WHEN pos = 'PD' then '1. ped'
                WHEN pos = 'BI' then '2. bicycle'
                WHEN pos in ('MO','PS','DR') THEN '3. mvo' END as "Type"
           ,0 as "Total Injuries"
           ,0 as "Severe Injuries"
           ,count(id_) as "Fatalities"
    FROM public.fatality_nycdot_current fat
    WHERE fat.segmentid in (SELECT segmentid FROM sam_fatalities_950)
    and date_part('year',acdate) between 2013 and 2017
    GROUP by pos

    UNION 

    -- All fatalities on intersections of stretch
    SELECT CASE WHEN pos = 'PD' then '1. ped'
                WHEN pos = 'BI' then '2. bicycle'
                WHEN pos in ('MO','PS','DR') then '3. mvo' end as "Type"
           ,0 as "Total Injuries"
           ,0 as "Severe Injuries"
           ,count(id_) as "Fatalities"
    FROM public.fatality_nycdot_current fat
    WHERE fat.nodeid::int in ( SELECT distinct nodeid::int 
                               FROM (SELECT nodeidFROM nodeid FROM sam_fatalities_950 
                                     UNION
                                     SELECT nodeidto nodeid FROM sam_fatalities_950) x
                             )
    and date_part('year',acdate) between 2013 and 2017
    GROUP by pos
) inj_sum
GROUP by "Type"
)

SELECT * FROM (
SELECT *
FROM data

union

SELECT 'Total' as "Type", "Total Injuries","Severe Injuries", "Fatalities", "KSI" 
FROM (SELECT 'Total' as Year, 
      sum(data."Total Injuries") "Total Injuries",
      sum(data."Severe Injuries") "Severe Injuries",
      sum(data."Fatalities") "Fatalities",
      sum(data."KSI") "KSI"
      FROM data
      ) tot
)inj_summary
ORDER BY "Type"

""")

In [15]:
inj_sum

,Type,Total Injuries,Severe Injuries,Fatalities,KSI
0,1. ped,16,0,1,1
1,2. bicycle,13,0,0,0
2,3. mvo,11,0,0,0
3,Total,40,0,1,1


### Fatalities by Mode

In [10]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-22 12:33:39


In [11]:
fat = cdb.dfquery("""

--Fatalities by mode-------------------------------------------------------------


SELECT CASE WHEN pos = 'PD' then 'ped'
            WHEN pos = 'BI' then 'bicycle'
            WHEN pos in ('MO','PS','DR') THEN 'mvo' END as "Type"
      ,count(CASE WHEN date_part('year',acdate) between 2013 and 2017 THEN id_ END) as "2013-2017 (5 Years)"
      ,count(CASE WHEN date_part('year',acdate)>=2013 and acdate < '07-07-2019'::date THEN id_ END) as "2013-07/07/2019 (7 Years)"
FROM public.fatality_nycdot_current fat
WHERE fat.segmentid in (SELECT segmentid FROM sam_fatalities_950)
GROUP by pos

UNION 

SELECT CASE WHEN pos = 'PD' THEN 'ped'
            WHEN pos = 'BI' THEN 'bicycle'
            WHEN pos in ('MO','PS','DR') THEN 'mvo' END as "Type"
      ,count(CASE WHEN date_part('year',acdate) between 2013 and 2017 THEN id_ END) as "2013-2017 (5 Years)"
      ,count(CASE WHEN date_part('year',acdate)>=2013 and acdate < '07-07-2019'::date THEN id_ END) as "2013-07/07/2019 (7 Years)"
FROM public.fatality_nycdot_current fat
WHERE fat.nodeid::int in (SELECT distinct nodeid::int 
                          FROM (SELECT nodeidFROM nodeid FROM sam_fatalities_950 
                                UNION
                                SELECT nodeidto nodeid FROM sam_fatalities_950) x)
GROUP BY pos

""") 


In [12]:
fat

,Type,2013-2017 (5 Years),2013-07/07/2019 (7 Years)
0,bicycle,0,0
1,ped,1,2


### Non-Fatal Injuries By Severity 

In [19]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-22 12:34:54


In [20]:
non_fat_sev = cdb.dfquery("""

--NON-FATAL INJURIES BY SEVERITY---------------------------------------------------


WITH data as(
SELECT * 
FROM (  SELECT 
        '1. A' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Total"
        FROM sam_crashes_950

        union

        SELECT 
        '2. B' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) as "Total"
        FROM sam_crashes_950

        union

        SELECT 
        '3. C' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) as "Total"
        FROM sam_crashes_950

        union

        SELECT 
        '4. Unknown' as Severity,
        sum(CASE WHEN accd_type_int = 1 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0)  end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 and coalesce(length(ext_of_inj::text),0) != num_of_inj then num_of_inj - coalesce(length(ext_of_inj::text),0)  end) "Motor Vehicle",
        sum(num_of_inj - coalesce(length(ext_of_inj::text),0)) as "Total"
        FROM sam_crashes_950

        )sev
        ORDER BY severity
)


SELECT * 
FROM ( SELECT * 
       FROM data

       UNION 

       SELECT '5. Total' as Severity, Pedestrian, Bicyclist, "Motor Vehicle", "Total"
       FROM (SELECT 'Total' as Year
            ,sum(data.Pedestrian) Pedestrian
            ,sum(data.Bicyclist) Bicyclist
            ,sum(data."Motor Vehicle") "Motor Vehicle"
            ,sum(data."Total") "Total"
            FROM data
              ) tot
        )non_fat_inj
ORDER BY severity




""")

In [15]:
non_fat_sev.fillna(0)

,severity,pedestrian,bicyclist,Motor Vehicle,Total
0,1. A,7,1,10,18
1,2. B,18,19,12,49
2,3. C,35,44,154,233
3,4. Unknown,1,1,6,8
4,5. Total,61,65,182,308


### Injuries by Year

In [21]:
inj_year = cdb.dfquery("""


--Injuries by year---------------------------------------------------



WITH data as (
SELECT 
case_yr::text as Year
,coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj end),0) "Pedestrian" 
,coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj end),0) "Bicyclist"
,coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj end),0) "Motor Vehicle"
FROM sam_crashes_950
GROUP by case_yr
order by case_yr
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM (SELECT 'Total' as Year, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                    ) tot
               )inj_year
ORDER BY year

""")


In [22]:
inj_year

,year,Pedestrian,Bicyclist,Motor Vehicle,total
0,2013,9,12,47,68
1,2014,14,12,51,77
2,2015,15,10,38,63
3,2016,10,20,20,50
4,2017,13,11,26,50
5,Total,61,65,182,308


### Injuries By Time of Day By Type

In [23]:
inj_tod = cdb.dfquery("""


--Injuries by time of day by type---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END as Time,
coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj END),0) "Pedestrian", 
coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj END),0) "Bicyclist",
coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj END),0) "Motor Vehicle"
FROM sam_crashes_950
GROUP by CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END
ORDER BY case WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END)

SELECT * FROM (SELECT data.*, 
                "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total,
                (round((("Pedestrian"+"Bicyclist"+"Motor Vehicle")::float/(SELECT sum(num_of_inj) FROM sam_crashes_950)::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE time != '9. Unknown'

                UNION 


                SELECT * , "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data.time != '8. Unknown/Indeterminate'
                and time = '9. Unknown'


                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM (SELECT 'Total' as Time, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                      ) tot
                )inj_tod
ORDER BY time


""")

In [24]:
inj_tod

,time,Pedestrian,Bicyclist,Motor Vehicle,total,Percent of Known Injuries
0,1. 00:00-03:00,1,3,8,12,3.9000
1,2. 03:00-06:00,7,1,11,19,6.2000
2,3. 06:00-09:00,10,6,17,33,10.7000
3,4. 09:00-12:00,5,5,27,37,12.0000
4,5. 12:00-15:00,6,8,38,52,16.9000
5,6. 15:00-18:00,10,18,38,66,21.4000
6,7. 18:00-21:00,11,15,18,44,14.3000
7,8. 21:00-24:00,10,9,21,40,13.0000
8,9. Unknown,1,0,4,5,N/A
9,Total,61,65,182,308,N/A


### Pedestrian Injuries By Control And Pedestrian Action

In [25]:
ped_inj_con = cdb.dfquery("""


--Pedestrian Injuries By Control and Pedestrian Action---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_950
WHERE accd_type_int = 1
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
              WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
              WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
              WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
              WHEN PED_LOC = '2' THEN '5. Midblock'
              WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
              ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " "

""")

In [26]:
ped_inj_con

,,Number of Injuries,Percent of Known Injuries
0,1. Signalized Intersection: Crossing WITH Signal,21,42.9000
1,2. Signalized Intersection: Crossing Against S...,5,10.2000
2,4. Other Actions/Uncontrolled Intersection,7,14.3000
3,5. Midblock,16,32.7000
4,7. Total Known,49,100.0000
5,8. Unknown/Indeterminate,12,N/A
6,Total,61,N/A


### Bicycle Injuries By Intersection Control and Bicyclist Action

In [27]:
bi_inj_con = cdb.dfquery("""


--Bicycle Injuries By Intersection Control and Bicyclist Action---------------------------------------------------



WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_950
WHERE accd_type_int = 2
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION


                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) bi_inj_con
ORDER BY " " 

""")

In [28]:
bi_inj_con

,,Number of Injuries,Percent of Known Injuries
0,1. Signalized Intersection: Crossing WITH Signal,5,16.7000
1,2. Signalized Intersection: Crossing Against S...,4,13.3000
2,3. Stop-Controlled Intersection / Crosswalk,3,10.0000
3,4. Other Actions/Uncontrolled Intersection,14,46.7000
4,5. Midblock,4,13.3000
5,7. Total Known,30,100.0000
6,8. Unknown/Indeterminate,35,N/A
7,Total,65,N/A


In [29]:
mvo_inj_coll = cdb.dfquery("""


--Motor Vehicle Injuries By Collision Type---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END as " ",
sum(num_of_inj) as "Number of Injuries"--, crashid
FROM sam_crashes_950
WHERE accd_type_int = 3
GROUP by CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END--, crashid
order by " "
)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = 'Unknown'

                UNION

                SELECT 
                '9. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " " 


"""
                          )

In [30]:
mvo_inj_coll

,,Number of Injuries,Percent of Known Injuries
0,1. LEFT Turn,28,18.2000
1,2. Right Turn,0,0.0000
2,3. Rear-End,31,20.1000
3,4. Right-Angle,36,23.4000
4,5. Sideswipe(Same Direction),9,5.8000
5,6. Head-on,0,0.0000
6,7. Sideswipe(Opposite Direction),0,0.0000
7,8. Other Known,50,32.5000
8,9. Total Known,154,100.0000
9,Total,182,N/A


### Injuries By Age Group

In [33]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-22 12:35:51


In [34]:
inj_age = cdb.dfquery("""


--Injuries By Age GROUP---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
     WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
     WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
     WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
     WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End "Age GROUP",
coalesce(sum(case when accd_type_int = 1 then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 then num_of_inj end),0) "Bicyclist" 
FROM sam_crashes_950
LEFT JOIN (SELECT nys_v.crashid, age 
           FROM (SELECT distinct crashid
                 FROM archive."2019_11_13_nysdot_vehicle"
                 WHERE age between 1 and 120
                 and veh_typ in ('5','6')
                 GROUP by crashid
                 HAVING count(crashid) =1
                 ) nys_v
           JOIN archive."2019_11_13_nysdot_vehicle" nys_v_age
           on nys_v.crashid = nys_v_age.crashid
           WHERE veh_typ in ('5','6')
          ) ages
on sam_crashes_950.crashid = ages.crashid
GROUP by CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
              WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
              WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
              WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
              WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End
ORDER BY "Age GROUP"
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist" Total
                FROM (SELECT 'Total' as Year 
                     ,sum(data."Pedestrian") "Pedestrian"
                     ,sum(data."Bicyclist") "Bicyclist"
                     FROM data
                      ) tot

               ) inj_age
                
ORDER BY "Age GROUP"

""")

In [35]:
inj_age

,Age GROUP,Pedestrian,Bicyclist,total
0,1. Children(1-17),9,13,22
1,2. Young Adults(18-29),11,21,32
2,3. Adults(30-64),21,22,43
3,4. Seniors(65-120),6,1,7
4,5. Unknown,14,8,22
5,Total,61,65,126


In [36]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-22 12:35:56


### Ped Injuries By Veh Action And Pre Action

In [37]:
ped_veh_actn = cdb.dfquery("""


WITH data as(
SELECT 
CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
     WHEN pre_accd_actn = '02' THEN '2. Right Turn'
     WHEN pre_accd_actn = '01' THEN '3. Going Straight'
     WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
     WHEN pre_accd_actn = '15' THEN '5. Backing'
     WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'   
     WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END as " ",
coalesce(sum(CASE WHEN ped_actn = '01' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing WITH Signal",
coalesce(sum(CASE WHEN ped_actn = '02' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing Against Signal",
coalesce(sum(CASE WHEN ped_actn = '03' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing No Signal Marked Crosswalk",
coalesce(sum(CASE WHEN ped_actn = '04' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing No Signal or Crosswalk",
coalesce(sum(CASE WHEN ped_actn not in ('01','02','03','04','??','YY','XX','ZZ') and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Others",
coalesce(sum(CASE WHEN ped_actn in ('??','YY','XX', 'ZZ') or single_ped.crashid is null or (num_of_inj>1 and single_ped.crashid is not null) THEN num_of_inj END),0) as "UnKnown"
FROM sam_crashes_950
LEFT JOIN (SELECT distinct crashid
         FROM archive."2019_11_13_nysdot_vehicle"
         WHERE veh_typ = '6'
         GROUP by crashid
         HAVING count(crashid) = 1) single_ped
     on sam_crashes_950.crashid = single_ped.crashid
LEFT JOIN (SELECT sv.crashid, sv2.pre_accd_actn
       FROM (SELECT distinct crashid--, pre_accd_actn
                 FROM archive."2019_11_13_nysdot_vehicle"
                 WHERE veh_typ not in ('5','6')		
                 GROUP by crashid--, pre_accd_actn
                 HAVING count(crashid) = 1)sv
         LEFT JOIN (SELECT distinct crashid, pre_accd_actn
               FROM archive."2019_11_13_nysdot_vehicle"
               WHERE veh_typ not in ('5','6')
               --WHERE pre_accd_actn not in ('??','YY','XX')
               ) sv2
            on sv.crashid = sv2.crashid) single_veh
    on sam_crashes_950.crashid = single_veh.crashid
WHERE accd_type_int = 1
GROUP by CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
          WHEN pre_accd_actn = '02' THEN '2. Right Turn'
              WHEN pre_accd_actn = '01' THEN '3. Going Straight'
          WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
          WHEN pre_accd_actn = '15' THEN '5. Backing'
          WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'     
          WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END
)


SELECT * FROM ( SELECT data.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM data

        UNION 

        SELECT tot.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM (SELECT 'Total' as " ", 
              sum(data."Crossing WITH Signal") "Crossing WITH Signal",
              sum(data."Crossing Against Signal") "Crossing Against Signal",
              sum(data."Crossing No Signal Marked Crosswalk") "Crossing No Signal Marked Crosswalk",
              sum(data."Crossing No Signal or Crosswalk") "Crossing No Signal or Crosswalk",
              sum(data."Others") "Others",
              sum(data."UnKnown") "UnKnown"
              FROM data
            ) tot

        ) ped_veh_actn
ORDER BY " "


""")

In [38]:
ped_veh_actn

,,Crossing WITH Signal,Crossing Against Signal,Crossing No Signal Marked Crosswalk,Crossing No Signal or Crosswalk,Others,UnKnown,total
0,1. Left Turn,10,2,1,1,0,4,18
1,2. Right Turn,6,0,0,0,0,5,11
2,3. Going Straight,2,4,2,4,4,2,18
3,4. Making U Turn,0,1,0,1,0,0,2
4,5. Backing,2,0,0,1,0,0,3
5,6. Other,0,0,0,0,2,0,2
6,7. Unknown,1,0,0,0,0,6,7
7,Total,21,7,3,7,6,17,61
